### 1. How to Select appropiate optimizer
### 2. No. of nodes in a layer
### 3. How to select no. of layers
### 4. ALL in one Model

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('diabetes.csv')

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.shape

(768, 9)

In [5]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [6]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [8]:
X = scaler.fit_transform(X)

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=1)

In [52]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [11]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = 'accuracy')

In [12]:
model.fit(X_train,y_train,batch_size=32,epochs= 100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 [==============================] - 1s 8ms/step - loss: 0.7622 - accuracy: 0.5879 - val_loss: 0.7166 - val_accuracy: 0.6104
Epoch 2/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6841 - accuracy: 0.6498 - val_loss: 0.6471 - val_accuracy: 0.6623
Epoch 3/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6254 - accuracy: 0.6889 - val_loss: 0.5969 - val_accuracy: 0.7078
Epoch 4/100
20/20 [==============================] - 0s 2ms/step - loss: 0.5840 - accuracy: 0.7248 - val_loss: 0.5614 - val_accuracy: 0.7597
Epoch 5/100
20/20 [==============================] - 0s 2ms/step - loss: 0.5537 - accuracy: 0.7410 - val_loss: 0.5352 - val_accuracy: 0.7857
Epoch 6/100
20/20 [==============================] - 0s 2ms/step - loss: 0.5312 - accuracy: 0.7687 - val_loss: 0.5187 - val_accuracy: 0.7857
Epoch 7/100
20/20 [==============================] - 0s 2ms/step - loss: 0.5160 - accuracy: 0.7720 - val_loss: 0.5060 - val_accuracy: 0.7922
Epoch 8/100
2

In [13]:
X_train.shape,y_train.shape


((614, 8), (614,))

In [14]:
import keras_tuner as kt

In [15]:
# Using Keras Tuner for selecting best optimizer

In [16]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))

    optimizer = hp.Choice('optimizer',values = ['adam', 'sgd', 'rmsprop', 'adadelta'])

    model.compile(optimizer=optimizer, loss = 'binary_crossentropy', metrics=['accuracy'])

    return model

In [17]:
tuner = kt.RandomSearch(build_model,
                       objective = 'val_accuracy',
                       max_trials = 5)

Reloading Tuner from .\untitled_project\tuner0.json


In [18]:
tuner.search(X_train,y_train, epochs = 10, validation_data=(X_test,y_test))

In [19]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [20]:
model = tuner.get_best_models(num_models=1)[0]

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
model.fit(X_train,y_train,batch_size=32,epochs = 100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4769 - accuracy: 0.7834 - val_loss: 0.4811 - val_accuracy: 0.7792
Epoch 8/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4732 - accuracy: 0.7801 - val_loss: 0.4791 - val_accuracy: 0.7857
Epoch 9/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4690 - accuracy: 0.7818 - val_loss: 0.4756 - val_accuracy: 0.7857
Epoch 10/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4651 - accuracy: 0.7801 - val_loss: 0.4757 - val_accuracy: 0.7792
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4625 - accuracy: 0.7834 - val_loss: 0.4754 - val_accuracy: 0.7727
Epoch 12/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4607 - accuracy: 0.7769 - val_loss: 0.4750 - val_accuracy: 0.7727
Epoch 13/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4586 - accuracy: 0.7785 - val_loss: 0.4749 - val_accuracy: 0.7727
Epoch 14/

In [31]:
def build_model(hp):
    model = Sequential()
    units= hp.Int('units',8,128)

    model.add(Dense(units=units,activation = 'relu', input_dim=8))
    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = 'accuracy')

    return model

In [32]:
tuner = kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=5,
                       directory='mydir',
                       project_name = 'CD1107_1')

In [33]:
tuner.search(X_train,y_train,epochs = 5, validation_data = (X_test,y_test) )

Trial 5 Complete [00h 00m 01s]
val_accuracy: 0.7727272510528564

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 05s


In [34]:
tuner.get_best_hyperparameters()[0].values

{'units': 87}

In [35]:
model = tuner.get_best_models(num_models=1)[0]

In [36]:
model.fit(X_train,y_train,batch_size=32,epochs = 100, initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4835 - accuracy: 0.7720 - val_loss: 0.4749 - val_accuracy: 0.8052
Epoch 8/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4717 - accuracy: 0.7720 - val_loss: 0.4712 - val_accuracy: 0.8117
Epoch 9/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4650 - accuracy: 0.7736 - val_loss: 0.4685 - val_accuracy: 0.8182
Epoch 10/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4592 - accuracy: 0.7785 - val_loss: 0.4684 - val_accuracy: 0.8182
Epoch 11/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4548 - accuracy: 0.7736 - val_loss: 0.4664 - val_accuracy: 0.8247
Epoch 12/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4531 - accuracy: 0.7752 - val_loss: 0.4642 - val_accuracy: 0.8117
Epoch 13/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4508 - accuracy: 0.7720 - val_loss: 0.4640 - val_accuracy: 0.8052
Epoch 14/

In [37]:
def build_model(hp):
    model = Sequential()
    
    model.add(Dense(87,activation='relu',input_dim = 8))

    for i in range(hp.Int('num_layers',min_value = 1, max_value = 10)):
        model.add(Dense(87,activation = 'relu'))

    model.add(Dense(1,activation = 'sigmoid'))
    model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

    return model

In [38]:
tuner = kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials = 3,
                       directory = 'mydir',
                       project_name = 'num_layers')

In [39]:
tuner.search(X_train,y_train,epochs = 5,validation_data = (X_test,y_test))

Trial 3 Complete [00h 00m 01s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 04s


In [40]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4}

In [41]:
model = tuner.get_best_models(num_models=1)[0]

In [42]:
model.fit(X_train,y_train, epochs = 100, initial_epoch=6, validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 9ms/step - loss: 0.4794 - accuracy: 0.7687 - val_loss: 0.4766 - val_accuracy: 0.7857
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4575 - accuracy: 0.7785 - val_loss: 0.5109 - val_accuracy: 0.7532
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4548 - accuracy: 0.7801 - val_loss: 0.4702 - val_accuracy: 0.7857
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4399 - accuracy: 0.7736 - val_loss: 0.4679 - val_accuracy: 0.7922
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4426 - accuracy: 0.7769 - val_loss: 0.4660 - val_accuracy: 0.7857
Epoch 12/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4345 - accuracy: 0.7818 - val_loss: 0.4719 - val_accuracy: 0.7857
Epoch 13/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4200 - accuracy: 0.8029 - val_loss: 0.4974 - val_accuracy: 0.7792
Epoch 14/

In [59]:
def build_model(hp):
    model = Sequential()
    counter = 0

    
    for i in range(hp.Int('num_layers',min_value = 1, max_value = 10)):

        if counter == 0:
            model.add(
                Dense(
                    hp.Int('units' + str(i),min_value = 8, max_value = 128,step = 8),
                    activation = hp.Choice('activation' + str(i),values = ['relu','tanh','sigmoid']),
                    input_dim = 8
                    )
                )
            model.add(Dropout(hp.Choice('dropout' + str(i), values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        else: 
            model.add(
                Dense(
                    hp.Int('units' + str(i),min_value = 8, max_value = 128,step = 8),
                    activation = hp.Choice('activation' + str(i),values = ['relu','tanh','sigmoid'])
                    )
                )
            model.add(Dropout(hp.Choice('dropout' + str(i), values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        counter+=1
    model.add(Dense(1,activation = 'sigmoid'))
    model.compile(optimizer= hp.Choice('optimizer',values = ['rmsprop','adam','sgd','ndam','adadelta']),loss = 'binary_crossentropy',metrics = ['accuracy'] )

    return model

In [60]:
tuner = kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=5,
                       directory = 'mydir',
                       project_name = 'all_2')

In [61]:
tuner.search(X_train,y_train,epochs = 5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 00s]

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 00m 08s


In [62]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 5,
 'units0': 120,
 'activation0': 'relu',
 'dropout0': 0.8,
 'optimizer': 'sgd',
 'units1': 8,
 'activation1': 'relu',
 'dropout1': 0.1,
 'units2': 8,
 'activation2': 'relu',
 'dropout2': 0.1,
 'units3': 8,
 'activation3': 'relu',
 'dropout3': 0.1,
 'units4': 8,
 'activation4': 'relu',
 'dropout4': 0.1}

In [63]:
model = tuner.get_best_models(num_models=1)[0]

In [64]:
model.fit(X_train,y_train,epochs = 200,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/200
20/20 [==============================] - 1s 8ms/step - loss: 0.6856 - accuracy: 0.5961 - val_loss: 0.6768 - val_accuracy: 0.6429
Epoch 8/200
20/20 [==============================] - 0s 3ms/step - loss: 0.6730 - accuracy: 0.6564 - val_loss: 0.6718 - val_accuracy: 0.6429
Epoch 9/200
20/20 [==============================] - 0s 3ms/step - loss: 0.6729 - accuracy: 0.6547 - val_loss: 0.6669 - val_accuracy: 0.6429
Epoch 10/200
20/20 [==============================] - 0s 3ms/step - loss: 0.6720 - accuracy: 0.6482 - val_loss: 0.6631 - val_accuracy: 0.6429
Epoch 11/200
20/20 [==============================] - 0s 3ms/step - loss: 0.6696 - accuracy: 0.6515 - val_loss: 0.6590 - val_accuracy: 0.6429
Epoch 12/200
20/20 [==============================] - 0s 3ms/step - loss: 0.6580 - accuracy: 0.6531 - val_loss: 0.6556 - val_accuracy: 0.6429
Epoch 13/200
20/20 [==============================] - 0s 3ms/step - loss: 0.6563 - accuracy: 0.6515 - val_loss: 0.6533 - val_accuracy: 0.6429
Epoch 14/